In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_horizVert_wavelet_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,181356.86,-2.85,-1.69,-1.13,-0.25,-0.20,0.06,-0.07,-0.01
3,-2.85,38768.01,-0.72,0.10,0.07,-0.01,-0.01,0.01,0.01
4,-1.69,-0.72,9780.55,0.03,0.07,-0.01,-0.02,-0.00,-0.00
5,-1.13,0.10,0.03,1797.86,-0.01,0.00,0.00,-0.01,-0.00
6,-0.25,0.07,0.07,-0.01,385.32,0.01,0.00,0.00,0.00
7,-0.20,-0.01,-0.01,0.00,0.01,101.35,0.00,0.00,0.00
8,0.06,-0.01,-0.02,0.00,0.00,0.00,34.48,0.00,-0.00
9,-0.07,0.01,-0.00,-0.01,0.00,0.00,0.00,9.88,-0.00
10,-0.01,0.01,-0.00,-0.00,0.00,0.00,-0.00,-0.00,1.44


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00003,-0.00004,-0.00006,-0.00003,-0.00005,0.00002,-0.00005,-0.00002
3,-0.00003,1.00000,-0.00004,0.00001,0.00002,-0.00001,-0.00001,0.00001,0.00005
4,-0.00004,-0.00004,1.00000,0.00001,0.00004,-0.00001,-0.00003,-0.00001,-0.00001
5,-0.00006,0.00001,0.00001,1.00000,-0.00001,0.00000,0.00001,-0.00006,-0.00005
6,-0.00003,0.00002,0.00004,-0.00001,1.00000,0.00004,0.00002,0.00002,0.00000
7,-0.00005,-0.00001,-0.00001,0.00000,0.00004,1.00000,0.00001,0.00006,0.00000
8,0.00002,-0.00001,-0.00003,0.00001,0.00002,0.00001,1.00000,0.00000,-0.00001
9,-0.00005,0.00001,-0.00001,-0.00006,0.00002,0.00006,0.00000,1.00000,-0.00004
10,-0.00002,0.00005,-0.00001,-0.00005,0.00000,0.00000,-0.00001,-0.00004,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

5.074778250615153

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.65741615e+09 1.64501153e+08 1.02448766e+07 3.37053136e+05
 1.48952925e+04 9.73562908e+02 9.99313429e+01 6.09487477e+00
 2.74728503e-31]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999585,-0.027974,-0.006721,-0.001236,-0.000265,-0.000070,-0.000023,-0.000007,-0.000001
1,0.027685,0.998847,-0.038630,-0.006594,-0.001409,-0.000371,-0.000126,-0.000036,-0.000005
2,0.007743,0.038189,0.998706,-0.031942,-0.006581,-0.001731,-0.000590,-0.000169,-0.000025
3,0.001648,0.007688,0.031306,0.998394,-0.045011,-0.011230,-0.003806,-0.001096,-0.000160
4,0.000420,0.001953,0.007750,0.043771,0.996251,-0.070513,-0.022146,-0.006289,-0.000914
5,0.000135,0.000626,0.002479,0.013517,0.066378,0.989126,-0.126666,-0.031135,-0.004475
6,0.000061,0.000286,0.001134,0.006159,0.028887,0.118470,0.979986,-0.155951,-0.019966
7,0.000027,0.000125,0.000495,0.002694,0.012549,0.047915,0.146218,0.976721,-0.148916
8,0.000008,0.000036,0.000146,0.000793,0.003688,0.014020,0.041221,0.143825,0.988638


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00041474, 0.00115285, 0.0012935 , 0.00160616, 0.00374851,
       0.01087358, 0.02001447, 0.02327927, 0.01136234])